In [319]:
import pandas as pd
import requests as re

+ В данном ноутбуке, я продемонстрирую обработку запросов в докере для методов fit, predict
+ Проверка, что фит отрабатывает в параллельном режиме
+ Для того, чтобы показать результативность остальных методов буду кидать запрос на сервер запущенный без докера, чтобы корректно показать файлы в директориях

##### Проверим работоспособность методов на игрушечных данных

In [357]:
req_fit = {
    "X": [[1,2],[3,4],[5,6]],
    "y": [1,3,7],
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"tmp_1"
    }
}

req_predict = {
    "X": [[1,2],[3,4],[5,6]],
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"tmp_1"
    }
}

In [361]:
res_fit = re.post(url='http://0.0.0.0:8000/fit',
              json=req_fit)
res_fit.text

'{"msg":"Поставлено на обучение, время - 18:19:46, свободных процессов 3, "}'

In [362]:
res_predict = re.post(url='http://0.0.0.0:8000/predict',
              json=req_predict)

res_predict

<Response [200]>

In [363]:
res_predict.text

'{"время сохранения модели":["18:19:47"],"predictions":[1.0,3.0,7.0]}'

##### Сделаем проверку на больших данных, заодно проверим параллельность обучения моделей

In [380]:
import numpy as np
import pandas as pd
X = np.random.randint(0,100,size=(1000000, 150))
y = np.random.randint(0,100,size=(1000000, 1))

X = X.tolist()
y = y.ravel().tolist()

/var/folders/4t/ss801yzj3_585ln7ynwmstgm0000gn/T/ipykernel_9456/2041206027.py:6: RuntimeWarning: coroutine 'ClientSession._request' was never awaited
  X = X.tolist()


In [377]:
## Cоздадим два тела запроса для обучения и предикта, закинем запросы на обучения одновременно и чекнем время

req_fit_1 = {
    "X": X,
    "y": y,
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"check_1"
    }
}

req_predict_1 = {
    "X": X,
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"check_1"
    }
}

req_fit_2 = {
    "X": X,
    "y": y,
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"check_2"
    }
}

req_predict_2 = {
    "X": X,
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"check_2"
    }
}

In [355]:
import asyncio
import aiohttp

async def test():
    async with aiohttp.ClientSession() as session:
        res_1 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_1)
        print(await res_1.json())
        res_2 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_2)
        print(await res_2.json())

loop = asyncio.get_event_loop()
task = loop.create_task(test())
loop.run_until_complete(task)

{'msg': 'Поставлено на обучение, время - 18:14:58, свободных процессов 3, '}
{'msg': 'Поставлено на обучение, время - 18:15:09, свободных процессов 2, '}


In [356]:
res_pred_1 = re.post(url='http://0.0.0.0:8000/predict',
              json=req_predict_1)
res_pred_2 = re.post(url='http://0.0.0.0:8000/predict',
              json=req_predict_2)

print(res_pred_1.json()['время сохранения модели'])
print(res_pred_2.json()['время сохранения модели'])

['18:15:12']
['18:15:23']


Видим, что вторая модель поставлена на обучение раньше, чем сохранилась первая. Что говорит о рабочей варианте многороцессорности сервера.

#### Демонстрация методов remove, remove_all

In [365]:
import os

print('изначальное состояние директории',os.listdir(path='/Users/danzan/Desktop/fastApiProject/output'))


remove_req = re.post(url='http://127.0.0.1:8000/remove',
              json={
    "model":"Tree",
    "task":"reg",
    "name":"tmp_1"
})

print('состояние директории после удаления файла tmp_1: ',os.listdir(path='/Users/danzan/Desktop/fastApiProject/output'))

remove_req = re.get(url='http://127.0.0.1:8000/remove_all')

print('состояние директории после удаления всех файлов:',os.listdir(path='/Users/danzan/Desktop/fastApiProject/output'))

изначальное состояние директории ['check_2.sav', 'check_1.sav', 'tmp_3.sav', 'tmp_2.sav', 'tmp_1.sav']
состояние директории после удаления файла tmp_1:  ['check_2.sav', 'check_1.sav', 'tmp_3.sav', 'tmp_2.sav']
состояние директории после удаления всех файлов: []


В реализации своего веб-сервера, я не стал использовать ручки load и unload, вместо этого я использую методы load и unload, внутри ручек
fit и predict. Не совсем понял целесообразность использования ручек load и unload

#### Демонстрация обработки краевых случаев
+ Обработка предикта с несуществующей моделью
+ обработка события, когда нет свободных ядер

In [370]:
req_predict = {
    "X": [[1,2],[3,4],[5,6]],
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"tmp_1"
    }
}

res_predict = re.post(url='http://0.0.0.0:8000/predict',
              json=req_predict)

res_predict.text

'{"detail":"Нет модели с таким именем на диске"}'

In [383]:
import asyncio
import aiohttp

import numpy as np
import pandas as pd
X = np.random.randint(0,100,size=(100, 50))
y = np.random.randint(0,100,size=(100, 1))

X = X.tolist()
y = y.ravel().tolist()

req_fit_ex = {
    "X": X,
    "y": y,
    "config":{
        "model":"Tree",
        "task":"reg",
        "name":"check_2"
    }
} 

async def test():
    async with aiohttp.ClientSession() as session:
        res_1 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_ex)
        print(await res_1.json())
        res_2 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_ex)
        print(await res_2.json())
        res_3 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_ex)
        print(await res_3.json())
        res_4 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_ex)
        print(await res_4.json())
        res_5 = await session.post(url='http://0.0.0.0:8000/fit', json=req_fit_ex)
        print(await res_5.json())

loop = asyncio.get_event_loop()
task = loop.create_task(test())
loop.run_until_complete(task)

{'msg': 'Поставлено на обучение, время - 18:48:26, свободных процессов 3, '}
{'msg': 'Поставлено на обучение, время - 18:48:26, свободных процессов 2, '}
{'msg': 'Поставлено на обучение, время - 18:48:26, свободных процессов 1, '}
{'msg': 'Поставлено на обучение, время - 18:48:26, свободных процессов 0, '}
{'detail': 'нет свободных процессов, попробуйте позже'}


In [379]:
# from sklearn.tree import DecisionTreeRegressor

# tt = DecisionTreeRegressor()
# tt.fit(X,y)

# from datetime import datetime
# import time

# start_time = datetime.now()

# tt = DecisionTreeRegressor()
# tt.fit(X,y)

# print(datetime.now() - start_time)


0:00:14.982659
